In [ ]:
import os
from google.colab import driveэ

os.environ['HF_HOME'] = 'E:/PyProjects/.cache/'

drive.mount('/content/drive')

In [ ]:
%pip install -q accelerate peft bitsandbytes transformers trl scipy sentencepiece

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
import json
import os
import pprint

train_data = []

for file in os.listdir("drive/MyDrive/out_json_modified"):
    try:
        outfile = open('drive/MyDrive/out_json_modified/' + file, encoding="utf-8")
        data = json.load(outfile)
        train_data.append(data)
    except Exception as e:
        pprint.pp(e)

print(len(train_data))

2518


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
# config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    # config.base_model_name_or_path,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
# model = PeftModel.from_pretrained(
#     model,
#     MODEL_NAME,
#     torch_dtype=torch.float16
# )
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
DEFAULT_MESSAGE_TEMPLATE = "<|im_start|>{role}\n{content}"
DEFAULT_RESPONSE_TEMPLATE = "<|im_end|>"
DEFAULT_SYSTEM_PROMPT =  "Ты - ассистент преподавателя, который оценивает научную работу. Как и в любой работе, в тексте есть цели и задачи работы. Цель обычно одна, а задач - несколько. Твоя задача просмотреть и проанализировать весь текст работы и оценить соответствие текста поставленной цели и задачам. Тебе нужно проверить, соответствует ли текст поставленной цели и задачам. Если цель или задачи отсутствуют - так и напиши, что задачи не найдены. \
Для этого следуй данному плану:\
    1. Определи в тексте цель и задачи;\
    2. Прочитай и проанализируй весь текст;\
    3. Оцени соответствие текста поставленной цели и задачам;\
    4. Сделай подробный отчет.\
\
Обрати внимание на следующие правила: \
  1. Не придумывай цели и задачи, которых нет в тексте: Тебе запрещено добавлять собственные интерпретации или домыслы. Твои выводы должны строго основываться на информации, представленной в тексте.\
  2. Если увидишь, что в тексте есть информация, которая не содержится в цели и задачах, выдели ее в отчете, как некорректные данные.\
  3. Поскольку задачи и цель могут формулироваться разными вариантами, учти небольшую погрешность в отклонениях.\
  4. Процесс определения соответствия текста цели и задачам должен быть систематичным и логичным. Прежде чем писать отчет, внимательно прочитай текст несколько раз, чтобы полностью понять его содержание и контекст. Используй ключевые слова и фразы, которые могут указывать на намерения или план действий.\
  5. Текст будет разбиваться по чанкам, поэтому анализировать нужно только после последнего чанка.\
  6. Последний чанк будет иметь строку: 'ПОСЛЕДНИЙ ЧАНК'.\
  8. Оценив текст на соответствие поставленной цели и задачам, укажи в отчете подробно, что сделано правильно, а что нет. В этой задаче постарайся как похвалить автора, так и дать какие-то рекомендации по исправлениям, если они требуются.\
  8. В последнем пункте отчета попробуй сделать численную оценку в процентах на соответствие цели и задачам и объясни, почему оценка именно такая.

class Conversation:

    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [ ]:
def chunk_text(text, chunk_size, overlap):
    if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
        raise ValueError("Некорректные параметры: размер чанка должен быть положительным числом, "
                         "нахлёст должен быть неотрицательным числом и меньше размера чанка.")

    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap

        if start >= text_length:
            break

    return chunks

In [ ]:
def prepare_answer(inpt):

    chunks = chunk_text(inpt, 8000, 0)

    output = ''

    for i in range(len(chunks)):
        chunk = chunks[i]

        if i == len(chunks) - 1:
          chunk = chunk + 'ПОСЛЕДНИЙ ЧАНК'

        conversation = Conversation()
        conversation.add_user_message(chunk)
        prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)

    return output

In [ ]:
# for i in range(1, len(train_data)):
#     try:
#       out = train_data[i]
#       text = out['all_text']
#       generated_data = prepare_answer(text)
#       out['result'] = generated_data

#       with open(f'drive/MyDrive/out_json_correct_content/bachelor_{i+1}.json', 'w+', encoding='utf-8') as result_file:
#         json.dump(out, result_file, indent=4, ensure_ascii=False)

#     except Exception as e:
#         raise e

from pathlib import Path
import json

output_dir = Path('drive/MyDrive/out_json_correct_content2')
output_dir.mkdir(parents=True, exist_ok=True)

for i in range(77, len(train_data)):
    try:
        out = train_data[i]
        text = out['all_text']
        generated_data = prepare_answer(text)

        with open(output_dir / f'bachelor_{i+1}.json', 'w+', encoding='utf-8') as result_file:
            json.dump({"result": generated_data}, result_file, indent=4, ensure_ascii=False)
    except Exception as e:
        print(f"Error processing element {i}: {e}")


In [ ]:
torch.cuda.empty_cache()